### Import modules and DataFrame

In [1]:
import pandas as pd
import pickle
import numpy

In [2]:
dfs = pickle.load(open('dfs_2000_0401_0630.p','rb'))

In [3]:
dfs.head()

,abtm,br,brn,brv,chn,ckey,co,coiso,ct,devm,...,tPld,tVH,tid,ts,ts2,type,uid,utype,v,yob
0,Release,3614101,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,Canada,CA,Coquitlam,WooriKooli W,...,49377,59880,4e7475b2-0b88-461f-9387-0a9b77251df8,1491004567535,2017-03-31T23:56:07.535Z,vh,WUCH155103440,user,0.9.1,Unset
1,Release,3328416,KOOLi_Player,2.0.6,,17943e6c6eec49cdb6,United States,US,Los Angeles,h3,...,72154,36047,7e588404-453f-4a45-a8aa-05b6b0ae5be4,1491004632602,2017-03-31T23:57:12.602Z,lBuf,14110963601665,user,0.9.1,Unset
2,Release,3587620,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Laguna Woods,WooriKooli W,...,69839,12022,f19a8414-64cd-418a-985e-61ae073d8915,1491004617448,2017-03-31T23:56:57.448Z,lBuf,WUCH155102826,user,0.9.1,Unset
3,Release,1373174,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,West Springfield,WooriKooli W,...,73567,39988,7efccc8b-436d-40bf-8bd5-fe45d08039ca,1491004547792,2017-03-31T23:55:47.792Z,mplyevent,WUCH161600388,user,0.9.1,Unset
4,Release,1322244,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Medfield,WooriKooli W,...,70656,16127,14bd5375-4a03-460c-b0aa-d73fe16628b7,1491004467751,2017-03-31T23:54:27.751Z,vh,WUCH155101681,user,0.9.1,Unset


In [4]:
#dfs.info()
dfs.shape

(182000, 54)

### Here's what needs to be done..
    1. Identify each user's last activity
    2. If the last activity falls in recent 30 days, then the user = not churned
    3. Otherwise(i.e. the last activity is earlier than 30 days), the user has churned
    
    #=> Which means that I need to deal with date..

### 'ts' and 'ts2' for date

In [5]:
from datetime import datetime, timedelta

In [6]:
# (optional)
# Pandas Series works too..
# before we convert 'ts2' column type from string to datetime
print dfs['ts2'][:1]; print type(dfs['ts2'][:1]) 
print '\n'

print 'extract the value only: ',dfs['ts2'][:1].values[0]
a_not_used = datetime.strptime(dfs['ts2'][:1].values[0],'%Y-%m-%dT%H:%M:%S.%fZ')
print '\n'
print a_not_used ; print type(a_not_used)

0    2017-03-31T23:56:07.535Z
Name: ts2, dtype: object
<class 'pandas.core.series.Series'>


extract the value only:  2017-03-31T23:56:07.535Z


2017-03-31 23:56:07.535000
<type 'datetime.datetime'>


In [7]:
# But we use this..
# First row
print type(dfs.ix[0,:]['ts2'])

<type 'unicode'>


In [8]:
a = datetime.strptime(dfs.ix[0,:]['ts2'],'%Y-%m-%dT%H:%M:%S.%fZ')
print a; print type(a)
# same as using Pandas Series, except it's shorter..

2017-03-31 23:56:07.535000
<type 'datetime.datetime'>


In [9]:
a.microsecond

535000

In [10]:
a.date()

datetime.date(2017, 3, 31)

In [11]:
a.time()

datetime.time(23, 56, 7, 535000)

In [12]:
a_str = datetime.strptime(dfs.ix[0,:]['ts2'],'%Y-%m-%dT%H:%M:%S.%fZ').strftime('%s')
print a_str
a_in_ms = a_str + str(a.microsecond/1000)
print 'time in milliseconds in ts2: ', a_in_ms
print 'time in milliseconds in ts : ', dfs.ix[0,:]['ts']

1491029767
time in milliseconds in ts2:  1491029767535
time in milliseconds in ts :  1491004567535


    Wait.. What's up with that? Possibly the timezone issue?

In [13]:
import pytz

In [14]:
def utc_to_each_tz(utc_dt, region):
    import pytz
    local_tz = pytz.timezone(region)
    local_dt = utc_dt.replace(tzinfo=pytz.utc).astimezone(local_tz)
    return local_tz.normalize(local_dt)

In [15]:
## First row comes from 'Coquitlam, Canada' (same timezone with LA)
a_in_ms_tz = utc_to_each_tz(a,'America/Los_Angeles').strftime('%s')  + str(a.microsecond/1000)
print 'time in milliseconds in ts2 (tz adjusted): ', a_in_ms_tz
print 'time in milliseconds in ts               : ', dfs.ix[0,:]['ts']
print 'Are they same? ',a_in_ms_tz == dfs.ix[0,:]['ts']
# 

time in milliseconds in ts2 (tz adjusted):  1491004567535
time in milliseconds in ts               :  1491004567535
Are they same?  True


    Ahh.. It was timezone issue 
    ts is in UTC
    We'll get rid of 'ts' anyway...

In [16]:
dfs.shape

(182000, 54)

In [17]:
# Dropping the column 'ts'
if 'ts' in dfs.columns:
    dfs = dfs.drop('ts',axis=1)

In [18]:
dfs.shape

(182000, 53)

In [19]:
print dfs.ix[0,:]['ts2']
print type(dfs.ix[0,:]['ts2'])
# it's string..

2017-03-31T23:56:07.535Z
<type 'unicode'>


### A special object will be easier for me to handle..

In [20]:
# Convert 'ts2' column type from string to datetime
if isinstance(dfs.ix[0,:]['ts2'], unicode):
    dfs['ts2'] = pd.to_datetime(dfs['ts2'])

In [21]:
# Converted into datetime object
print dfs.ix[0,:]['ts2']
print type(dfs.ix[0,:]['ts2'])

2017-03-31 23:56:07.535000
<class 'pandas.tslib.Timestamp'>


### Also, we'll match the date type

In [22]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182000 entries, 0 to 181999
Data columns (total 53 columns):
abtm        182000 non-null object
br          167826 non-null object
brn         140477 non-null object
brv         140477 non-null object
chn         182000 non-null object
ckey        182000 non-null object
co          182000 non-null object
coiso       182000 non-null object
ct          182000 non-null object
devm        140477 non-null object
devv        140477 non-null object
drt         56448 non-null object
eps         182000 non-null object
errmsg      2262 non-null object
gen         173329 non-null object
img         9289 non-null object
isp         182000 non-null object
lang        182000 non-null object
live        182000 non-null object
lrg         182000 non-null object
mplyevnt    41523 non-null object
mpv         182000 non-null object
mvcp        182000 non-null object
mvctg       182000 non-null object
mvid        182000 non-null object
mvrt        182000 n

In [23]:
# Convert string into int

print dfs.ix[0,:]['tPld'] # player loading time
print type(dfs.ix[0,:]['tPld'])
print ''
print dfs.ix[0,:]['tIBuf'] # Initial Buffering time
print type(dfs.ix[0,:]['tIBuf'])
print ''
print dfs.ix[0,:]['tBuf'] # Re-buffering time
print type(dfs.ix[0,:]['tBuf'])
print ''
print dfs.ix[0,:]['tLBuf'] # Long buffering time
print type(dfs.ix[0,:]['tLBuf'])
print ''
print dfs.ix[0,:]['tVH'] # Viewing Time (in a minute)
print type(dfs.ix[0,:]['tVH'])


# what to convert
str_to_int_lst = ['tPld','tIBuf','tBuf','tLBuf','tVH']

49377
<type 'unicode'>

6
<type 'unicode'>

821
<type 'unicode'>

5003.0
<type 'numpy.float64'>

59880
<type 'unicode'>


In [24]:
def str_to_int_ftn(column_name):
    dfs[column_name] = dfs[column_name].fillna(0).astype(int)

In [25]:
for column_name in str_to_int_lst:
    str_to_int_ftn(column_name)

In [26]:
dfs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 182000 entries, 0 to 181999
Data columns (total 53 columns):
abtm        182000 non-null object
br          167826 non-null object
brn         140477 non-null object
brv         140477 non-null object
chn         182000 non-null object
ckey        182000 non-null object
co          182000 non-null object
coiso       182000 non-null object
ct          182000 non-null object
devm        140477 non-null object
devv        140477 non-null object
drt         56448 non-null object
eps         182000 non-null object
errmsg      2262 non-null object
gen         173329 non-null object
img         9289 non-null object
isp         182000 non-null object
lang        182000 non-null object
live        182000 non-null object
lrg         182000 non-null object
mplyevnt    41523 non-null object
mpv         182000 non-null object
mvcp        182000 non-null object
mvctg       182000 non-null object
mvid        182000 non-null object
mvrt        182000 n

### Identify each user's last activity

In [27]:
print 'The number of unique users:',len(dfs['uid'].unique())
print type(dfs['uid'].unique())
unique_uid = dfs['uid'].unique()

The number of unique users: 4197
<type 'numpy.ndarray'>


In [28]:
##unique uid looks like..
unique_uid[:10]

array([u'WUCH155103440', u'14110963601665', u'WUCH155102826',
       u'WUCH161600388', u'WUCH155101681', u'WUCH155102162',
       u'WUCH155101131', u'WUCH155101270', u'WUCH155102245',
       u'WUCH155102068'], dtype=object)

In [29]:
# to get the last activity for unique users
dfs_last = dfs.groupby('uid').last()
print dfs_last.shape

(4197, 52)


In [30]:
# This DataFrame shows the last activity of each unique user'
dfs_last.head()

,abtm,br,brn,brv,chn,ckey,co,coiso,ct,devm,...,tIBuf,tLBuf,tPld,tVH,tid,ts2,type,utype,v,yob
uid,,,,,,,,,,,,,,,,,,,,,
13110946900174,Release,3425963,KOOLi_Player,1.0.11,,17943e6c6eec49cdb6,United States,US,Santa Ana,WooriKooli W,...,0,5004,0,23978,0ceeb774-b2ff-4fae-a8e4-e26b09862c98,2017-06-25 23:53:40.532,vh,user,0.9.1,Unset
14110954700477,Release,5473981,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,China,CN,Guangzhou,WooriKooli W,...,0,0,0,0,7601f004-5675-4369-8403-7bcc26574bd2,2017-06-05 23:51:55.457,mplyevent,user,0.9.1,Unset
14110962300001,Release,3577044,KOOLi_Player,2.0.7,,17943e6c6eec49cdb6,United States,US,Teaneck,h3,...,0,5003,0,59858,aa8c169e-ca28-4463-b7eb-6f3a92ab85d4,2017-06-11 23:49:44.582,vh,user,0.9.1,Unset
14110962300002,Release,3428368,KOOLi_Player,1.0.11,,17943e6c6eec49cdb6,United States,US,Fort Lee,WooriKooli W,...,0,0,0,0,384d9f1f-e366-4ba0-adbd-1ae5c3c2a267,2017-06-27 23:57:21.619,mplyevent,user,0.9.1,Unset
14110962300003,Release,3530742,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Fort Lee,WooriKooli W,...,0,5003,0,0,611de4d6-8403-4e2d-a0f4-8986773a5350,2017-04-28 23:46:52.640,vh,user,0.9.1,Unset


In [31]:
# The number of rows in dfs_last eqauls to the number of unique users
dfs_last.shape[0] == len(unique_uid)

True

In [32]:
# (Optional)
# to get the first activity for unique users
dfs_first = dfs.groupby('uid').first()
print dfs_first.shape

(4197, 52)


In [33]:
# (Optional)
dfs_first.head()

,abtm,br,brn,brv,chn,ckey,co,coiso,ct,devm,...,tIBuf,tLBuf,tPld,tVH,tid,ts2,type,utype,v,yob
uid,,,,,,,,,,,,,,,,,,,,,
13110946900174,Release,3649390,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Irvine,WooriKooli W,...,0,0,0,60002,4695ba6e-5e33-49ce-9210-4977385df779,2017-04-02 23:56:54.163,vh,user,0.9.1,Unset
14110954700477,Release,899884,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,China,CN,Guangzhou,WooriKooli W,...,0,5004,0,7956,7601f004-5675-4369-8403-7bcc26574bd2,2017-06-05 23:51:16.066,upImg,user,0.9.1,Unset
14110962300001,Release,3464687,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Teaneck,WooriKooli W,...,0,0,0,0,e2272e69-00dc-459d-9d67-31c50b4d6454,2017-04-06 23:56:43.105,vh,user,0.9.1,Unset
14110962300002,Release,5510220,KOOLi_Player,2.0.7,,17943e6c6eec49cdb6,United States,US,Fort Lee,h3,...,0,0,0,0,adb4c1dd-5817-4df8-97ef-7789624e0bdc,2012-03-29 07:36:03.186,mplyevent,user,0.9.1,Unset
14110962300003,Release,3453604,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Fort Lee,WooriKooli W,...,0,5005,0,0,73ce457f-45a3-4f0e-808b-bf61e174456e,2017-04-15 23:47:47.132,vh,user,0.9.1,Unset


In [34]:
# (Optional)
# Sanity check
dfs_first['ts2'] <= dfs_last['ts2']

uid
13110946900174    True
14110954700477    True
14110962300001    True
14110962300002    True
14110962300003    True
14110962300004    True
14110962300005    True
14110962300006    True
14110962300007    True
14110962300008    True
14110962300010    True
14110962300011    True
14110962300012    True
14110962300013    True
14110962300014    True
14110962300015    True
14110962300017    True
14110962300018    True
14110962300019    True
14110962300020    True
14110962300021    True
14110962300022    True
14110962300023    True
14110962300025    True
14110962300026    True
14110962300027    True
14110962300028    True
14110962300029    True
14110962300030    True
14110962300031    True
                  ... 
WUCH161600453     True
WUCH161600454     True
WUCH161600455     True
WUCH161600456     True
WUCH161600457     True
WUCH161600458     True
WUCH161600459     True
WUCH161600460     True
WUCH161600463     True
WUCH161600464     True
WUCH161600465     True
WUCH161600466     True
WUCH161

### See If the last activity falls in recent 30 days or not

In [35]:
dfs_last.head()

,abtm,br,brn,brv,chn,ckey,co,coiso,ct,devm,...,tIBuf,tLBuf,tPld,tVH,tid,ts2,type,utype,v,yob
uid,,,,,,,,,,,,,,,,,,,,,
13110946900174,Release,3425963,KOOLi_Player,1.0.11,,17943e6c6eec49cdb6,United States,US,Santa Ana,WooriKooli W,...,0,5004,0,23978,0ceeb774-b2ff-4fae-a8e4-e26b09862c98,2017-06-25 23:53:40.532,vh,user,0.9.1,Unset
14110954700477,Release,5473981,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,China,CN,Guangzhou,WooriKooli W,...,0,0,0,0,7601f004-5675-4369-8403-7bcc26574bd2,2017-06-05 23:51:55.457,mplyevent,user,0.9.1,Unset
14110962300001,Release,3577044,KOOLi_Player,2.0.7,,17943e6c6eec49cdb6,United States,US,Teaneck,h3,...,0,5003,0,59858,aa8c169e-ca28-4463-b7eb-6f3a92ab85d4,2017-06-11 23:49:44.582,vh,user,0.9.1,Unset
14110962300002,Release,3428368,KOOLi_Player,1.0.11,,17943e6c6eec49cdb6,United States,US,Fort Lee,WooriKooli W,...,0,0,0,0,384d9f1f-e366-4ba0-adbd-1ae5c3c2a267,2017-06-27 23:57:21.619,mplyevent,user,0.9.1,Unset
14110962300003,Release,3530742,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Fort Lee,WooriKooli W,...,0,5003,0,0,611de4d6-8403-4e2d-a0f4-8986773a5350,2017-04-28 23:46:52.640,vh,user,0.9.1,Unset


In [36]:
dfs_last['ts2'] <= '2017-05-31'

uid
13110946900174    False
14110954700477    False
14110962300001    False
14110962300002    False
14110962300003     True
14110962300004    False
14110962300005    False
14110962300006    False
14110962300007    False
14110962300008    False
14110962300010    False
14110962300011    False
14110962300012    False
14110962300013    False
14110962300014    False
14110962300015    False
14110962300017    False
14110962300018    False
14110962300019    False
14110962300020    False
14110962300021    False
14110962300022    False
14110962300023     True
14110962300025    False
14110962300026    False
14110962300027    False
14110962300028    False
14110962300029     True
14110962300030    False
14110962300031    False
                  ...  
WUCH161600453      True
WUCH161600454     False
WUCH161600455     False
WUCH161600456      True
WUCH161600457     False
WUCH161600458     False
WUCH161600459      True
WUCH161600460     False
WUCH161600463     False
WUCH161600464     False
WUCH16160046

In [37]:
# Using boolen mask
# Add one more column 'churn'
dfs_last['churn'] = (dfs_last['ts2'] <= '2017-05-31')

In [38]:
# (Optional)
# Instead of hard code the date
# We could use 'N days ago from now'
print datetime.now() 
print datetime.now() - timedelta(days=30) # crietia for churn

2017-08-09 17:38:22.048205
2017-07-10 17:38:22.048375


In [39]:
dfs_last.shape
# (4197, 52) => (4197,53)

(4197, 53)

In [40]:
dfs_last.head()

,abtm,br,brn,brv,chn,ckey,co,coiso,ct,devm,...,tLBuf,tPld,tVH,tid,ts2,type,utype,v,yob,churn
uid,,,,,,,,,,,,,,,,,,,,,
13110946900174,Release,3425963,KOOLi_Player,1.0.11,,17943e6c6eec49cdb6,United States,US,Santa Ana,WooriKooli W,...,5004,0,23978,0ceeb774-b2ff-4fae-a8e4-e26b09862c98,2017-06-25 23:53:40.532,vh,user,0.9.1,Unset,False
14110954700477,Release,5473981,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,China,CN,Guangzhou,WooriKooli W,...,0,0,0,7601f004-5675-4369-8403-7bcc26574bd2,2017-06-05 23:51:55.457,mplyevent,user,0.9.1,Unset,False
14110962300001,Release,3577044,KOOLi_Player,2.0.7,,17943e6c6eec49cdb6,United States,US,Teaneck,h3,...,5003,0,59858,aa8c169e-ca28-4463-b7eb-6f3a92ab85d4,2017-06-11 23:49:44.582,vh,user,0.9.1,Unset,False
14110962300002,Release,3428368,KOOLi_Player,1.0.11,,17943e6c6eec49cdb6,United States,US,Fort Lee,WooriKooli W,...,0,0,0,384d9f1f-e366-4ba0-adbd-1ae5c3c2a267,2017-06-27 23:57:21.619,mplyevent,user,0.9.1,Unset,False
14110962300003,Release,3530742,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Fort Lee,WooriKooli W,...,5003,0,0,611de4d6-8403-4e2d-a0f4-8986773a5350,2017-04-28 23:46:52.640,vh,user,0.9.1,Unset,True


In [41]:
churn_usr_df = dfs_last[dfs_last['churn']==True]

In [42]:
churn_usr_df.shape

(708, 53)

In [43]:
churn_usr_df

,abtm,br,brn,brv,chn,ckey,co,coiso,ct,devm,...,tLBuf,tPld,tVH,tid,ts2,type,utype,v,yob,churn
uid,,,,,,,,,,,,,,,,,,,,,
14110962300003,Release,3530742,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Fort Lee,WooriKooli W,...,5003,0,0,611de4d6-8403-4e2d-a0f4-8986773a5350,2017-04-28 23:46:52.640,vh,user,0.9.1,Unset,True
14110962300023,Release,3499534,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Madison,WooriKooli W,...,0,0,0,faeffc44-57e8-4918-992e-d675ced3e774,2017-04-20 23:48:42.502,vh,user,0.9.1,Unset,True
14110962300029,Release,2547806,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Sacramento,WooriKooli W,...,0,0,0,f78cef04-1602-4709-95cc-3d8872c57bbe,2017-04-02 23:57:22.418,vh,user,0.9.1,Unset,True
14110962300038,Release,3468135,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Burke,WooriKooli W,...,0,0,0,80489625-144a-4f7a-9253-82c2a18e8d66,2017-04-20 23:49:14.612,lBuf,user,0.9.1,Unset,True
14110962300042,Release,3598791,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Chino Hills,WooriKooli W,...,0,0,0,6e096619-d741-4ffd-b100-b04002c87f69,2017-04-28 23:47:26.992,vh,user,0.9.1,Unset,True
14110962300051,Release,3716777,KOOLi_Player,2.0.7,,17943e6c6eec49cdb6,United States,US,Buena Park,h3,...,0,0,0,5ff978f1-ab0e-4105-9b4b-c7356c79f071,2017-04-24 23:50:48.936,vh,user,0.9.1,Unset,True
14110962300069,Release,3580124,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Los Angeles,WooriKooli W,...,0,0,0,455f8f79-03e2-4c0e-8dab-7bd78088e4f8,2017-04-10 23:52:44.436,mplyevent,user,0.9.1,Unset,True
14110962300073,Release,2180614,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Santa Rosa,WooriKooli W,...,0,0,59866,5259d3e8-35e7-4fde-b614-615242bc6de5,2017-04-09 23:45:31.087,vh,user,0.9.1,Unset,True
14110962300074,Release,3619018,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Long Beach,WooriKooli W,...,5003,0,60031,c19ceef5-3eb4-4cff-83cc-5a0af213fdf9,2017-04-08 23:46:09.788,vh,user,0.9.1,Unset,True


In [44]:
print 'Ratio of churn users/unique users: ',\
    float(churn_usr_df.shape[0])/dfs_last.shape[0]

Ratio of churn users/unique users:  0.168691922802


In [45]:
# churn user list
churn_usr_lst = list(churn_usr_df.index)
print len(churn_usr_lst)

708


In [46]:
# churn user list looks like..
churn_usr_lst[:10]

[u'14110962300003',
 u'14110962300023',
 u'14110962300029',
 u'14110962300038',
 u'14110962300042',
 u'14110962300051',
 u'14110962300069',
 u'14110962300073',
 u'14110962300074',
 u'14110962300077']

### mark churn users in original DataFrame dfs 

In [47]:
dfs['uid'].isin(churn_usr_lst)

0         False
1         False
2          True
3         False
4         False
5         False
6         False
7         False
8         False
9          True
10        False
11        False
12        False
13         True
14        False
15        False
16        False
17        False
18        False
19        False
20         True
21        False
22         True
23        False
24        False
25        False
26        False
27        False
28        False
29        False
          ...  
181970    False
181971    False
181972    False
181973    False
181974    False
181975    False
181976    False
181977    False
181978    False
181979    False
181980    False
181981    False
181982    False
181983    False
181984    False
181985    False
181986    False
181987    False
181988    False
181989    False
181990    False
181991    False
181992    False
181993    False
181994    False
181995    False
181996    False
181997    False
181998    False
181999    False
Name: uid, dtype: bool

In [48]:
print dfs.ix[2,:]['uid']
print dfs.ix[2,:]['uid'] in churn_usr_lst

WUCH155102826
True


In [49]:
churn_usr_lst[470:480]

[u'WUCH155102778',
 u'WUCH155102780',
 u'WUCH155102804',
 u'WUCH155102812',
 u'WUCH155102816',
 u'WUCH155102818',
 u'WUCH155102819',
 u'WUCH155102822',
 u'WUCH155102826',
 u'WUCH155102838']

In [50]:
# Using boolen mask
# Add one more column 'churn'
dfs['churn'] = dfs['uid'].isin(churn_usr_lst) 
print dfs.shape
# (182000, 53) => (182000, 54)

(182000, 54)


In [51]:
dfs.head()

,abtm,br,brn,brv,chn,ckey,co,coiso,ct,devm,...,tPld,tVH,tid,ts2,type,uid,utype,v,yob,churn
0,Release,3614101,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,Canada,CA,Coquitlam,WooriKooli W,...,49377,59880,4e7475b2-0b88-461f-9387-0a9b77251df8,2017-03-31 23:56:07.535,vh,WUCH155103440,user,0.9.1,Unset,False
1,Release,3328416,KOOLi_Player,2.0.6,,17943e6c6eec49cdb6,United States,US,Los Angeles,h3,...,72154,36047,7e588404-453f-4a45-a8aa-05b6b0ae5be4,2017-03-31 23:57:12.602,lBuf,14110963601665,user,0.9.1,Unset,False
2,Release,3587620,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Laguna Woods,WooriKooli W,...,69839,12022,f19a8414-64cd-418a-985e-61ae073d8915,2017-03-31 23:56:57.448,lBuf,WUCH155102826,user,0.9.1,Unset,True
3,Release,1373174,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,West Springfield,WooriKooli W,...,73567,39988,7efccc8b-436d-40bf-8bd5-fe45d08039ca,2017-03-31 23:55:47.792,mplyevent,WUCH161600388,user,0.9.1,Unset,False
4,Release,1322244,KOOLi_Player,1.0.10,,17943e6c6eec49cdb6,United States,US,Medfield,WooriKooli W,...,70656,16127,14bd5375-4a03-460c-b0aa-d73fe16628b7,2017-03-31 23:54:27.751,vh,WUCH155101681,user,0.9.1,Unset,False


In [52]:
pickle.dump(dfs, open("dfs_churn_marked.p","wb"))